In [20]:
import pandas as pd
import torch
import numpy 

In [30]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [2]:
task_1a_dataframe = pandas.read_csv('task_1a_dataset.csv')

In [3]:
df = task_1a_dataframe
df.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2012,Bangalore,3,37,Male,No,0,0
1,Masters,2017,New Delhi,2,28,Male,No,4,0
2,Bachelors,2017,New Delhi,2,36,Male,No,3,0
3,Bachelors,2015,Bangalore,3,27,Male,Yes,5,0
4,Bachelors,2017,Bangalore,3,29,Male,No,4,0


In [6]:
df.isnull().sum()

Education                    0
JoiningYear                  0
City                         0
PaymentTier                  0
Age                          0
Gender                       0
EverBenched                  0
ExperienceInCurrentDomain    0
LeaveOrNot                   0
dtype: int64

In [18]:
import pandas as pd

# Assuming your DataFrame is named 'df'
# List of categorical columns to one-hot encode
categorical_columns = ['Education', 'PaymentTier']

# Apply one-hot encoding to all categorical columns
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# Encode binary categorical columns ('Gender', 'EverBenched', 'LeaveOrNot') using LabelEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
binary_categorical_columns = ['Age', 'JoiningYear', 'City', 'Gender', 'EverBenched', 'LeaveOrNot']
for column in binary_categorical_columns:
    df_encoded[column] = label_encoder.fit_transform(df_encoded[column])

# Display the encoded DataFrame
df_encoded = df_encoded.astype(int)
print(df_encoded)

      JoiningYear  City  Age  Gender  EverBenched  ExperienceInCurrentDomain  \
0               0     0   15       1            0                          0   
1               5     1    6       1            0                          4   
2               5     1   14       1            0                          3   
3               3     0    5       1            1                          5   
4               5     0    7       1            0                          4   
...           ...   ...  ...     ...          ...                        ...   
4628            1     0    4       0            0                          4   
4629            1     2   15       1            0                          2   
4630            6     1    5       1            0                          5   
4631            0     0    8       1            1                          2   
4632            3     0   11       1            1                          4   

      LeaveOrNot  Education_Bachelors  

In [19]:
df_encoded.columns

Index(['JoiningYear', 'City', 'Age', 'Gender', 'EverBenched',
       'ExperienceInCurrentDomain', 'LeaveOrNot', 'Education_Bachelors',
       'Education_Masters', 'Education_PHD', 'PaymentTier_1', 'PaymentTier_2',
       'PaymentTier_3'],
      dtype='object')

In [24]:
def data_preprocessing(task_1a_dataframe):
    df = task_1a_dataframe

    

    # Assuming your DataFrame is named 'df'
    # List of categorical columns to one-hot encode
    categorical_columns = ['Education', 'PaymentTier']
    
    # Apply one-hot encoding to all categorical columns
    df_encoded = pd.get_dummies(df, columns=categorical_columns)
    
    # Encode binary categorical columns ('Gender', 'EverBenched', 'LeaveOrNot') using LabelEncoder
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    binary_categorical_columns = ['Age', 'JoiningYear', 'City', 'Gender', 'EverBenched', 'LeaveOrNot']
    for column in binary_categorical_columns:
        df_encoded[column] = label_encoder.fit_transform(df_encoded[column])
    
    # Display the encoded DataFrame
    df_encoded = df_encoded.astype(int)
    

    return df_encoded

In [32]:
df_encoded.shape

(4633, 13)

In [48]:
def identify_features_and_targets(encoded_dataframe):
    features = encoded_dataframe.drop(columns=['LeaveOrNot'])
    target = encoded_dataframe['LeaveOrNot']

    features_and_targets = [features, target]

    return features_and_targets

In [31]:
def load_as_tensors(features_and_targets):
    [X, y] = features_and_targets
    X = X.values
    y = y.values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    X_train_tensor = torch.FloatTensor(X_train)
    X_test_tensor = torch.FloatTensor(X_test)
    y_train_tensor = torch.LongTensor(y_train)
    y_test_tensor = torch.LongTensor(y_test)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    batch_size = 32  # You can adjust the batch size as needed
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    tensors_and_iterable_training_data = [X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader]

    return tensors_and_iterable_training_data

# Example usage:
# features_and_targets = [X, y]  # Replace X and y with your actual data
# tensors_and_iterable_training_data = load_as_tensors(features_and_targets)

In [40]:
class Salary_Predictor(nn.Module):
    def __init__(self, input_features=12, hidden1=20, hidden2=20, output_features=2):
        super(Salary_Predictor, self).__init__()
        self.fc1 = nn.Linear(input_features, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, output_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [58]:
def model_loss_function():
    '''
    Define the loss function for the model
    '''
    loss_function = nn.CrossEntropyLoss()

    return loss_function
def model_optimizer(model):

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01) #learning rate
    return optimizer

def model_number_of_epochs():
    number_of_epochs=30
    return number_of_epochs

In [44]:
def training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer):
    training_data = TensorDataset(tensors_and_iterable_training_data[0],tensors_and_iterable_training_data[2])
    training_loader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)

    #validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=True)
    for epoch in range(number_of_epochs):
        running_loss = 0.0
        for i, data in enumerate(training_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # print(i)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            inputs = inputs.float() #converting into float 
            outputs = model(inputs)
            loss = loss_function(outputs.squeeze(), labels)
            # Backward pass
            loss.backward()
            # Update model parameters
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            average_loss = running_loss / len(training_loader)
        
        # Print epoch-wise loss
        print(f"Epoch {epoch+1}/{number_of_epochs}: Loss = {average_loss}")
            # if i % 2000 == 1999:    # print every 2000 mini-batches
            #     print('[%d, %5d] loss: %.3f' %
            #           (epoch + 1, i + 1, running_loss / 2000))
            #     running_loss = 0.0
    return model

def validation_function(trained_model, tensors_and_iterable_training_data):
    # Set the model to evaluation mode
    trained_model.eval()

    # Unpack the input arguments
    validation_data = tensors_and_iterable_training_data[1]
    validation_labels = tensors_and_iterable_training_data[3]
    # validation_dataset = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=True)
    # validation_data, validation_labels, validation_dataset = tensors_and_iterable_training_data

    # Create a DataLoader for the validation dataset
    # validation_loader = torch.utils.data.DataLoader(validation_dataset)

    # Move the validation data to the device (e.g., GPU) if available
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # validation_data = validation_data.to(device)
    # validation_labels = validation_labels.to(device)

    # Calculate the predictions of the trained model on the validation data
    with torch.no_grad():
        predictions = trained_model(validation_data)

    # Convert the predicted probabilities to class labels
    _, predicted_labels = torch.max(predictions, 1)

    # Calculate the accuracy of the model
    correct_predictions = (predicted_labels == validation_labels).sum().item()
    total_predictions = len(validation_labels)
    model_accuracy = correct_predictions / total_predictions

    return model_accuracy

In [ ]:
def training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer):
	'''
	Purpose:
	---
	All the required parameters for training are passed to this function.

	Input Arguments:
	---
	1. `model`: An object of the 'Salary_Predictor' class
	2. `number_of_epochs`: For training the model
	3. `tensors_and_iterable_training_data`: list containing training and validation data tensors 
											 and iterable dataset object of training tensors
	4. `loss_function`: Loss function defined for the model
	5. `optimizer`: Optimizer defined for the model

	Returns:
	---
	trained_model

	Example call:
	---
	trained_model = training_function(model, number_of_epochs, iterable_training_data, loss_function, optimizer)

	'''	
	#################	ADD YOUR CODE HERE	##################
	
	##########################################################

	return trained_model

In [ ]:
def validation_function(trained_model, tensors_and_iterable_training_data):
	'''
	Purpose:
	---
	This function will utilise the trained model to do predictions on the
	validation dataset. This will enable us to understand the accuracy of
	the model.

	Input Arguments:
	---
	1. `trained_model`: Returned from the training function
	2. `tensors_and_iterable_training_data`: list containing training and validation data tensors 
											 and iterable dataset object of training tensors

	Returns:
	---
	model_accuracy: Accuracy on the validation dataset

	Example call:
	---
	model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)

	'''	
	#################	ADD YOUR CODE HERE	##################

	##########################################################

	return model_accuracy

In [63]:
if __name__ == "__main__":

	# reading the provided dataset csv file using pandas library and 
	# converting it to a pandas Dataframe
	task_1a_dataframe = pandas.read_csv('task_1a_dataset.csv')

	# data preprocessing and obtaining encoded data
	encoded_dataframe = data_preprocessing(task_1a_dataframe)

	# selecting required features and targets
	features_and_targets = identify_features_and_targets(encoded_dataframe)

	# obtaining training and validation data tensors and the iterable
	# training data object
	tensors_and_iterable_training_data = load_as_tensors(features_and_targets)
	
	# model is an instance of the class that defines the architecture of the model
	model = Salary_Predictor()

	# obtaining loss function, optimizer and the number of training epochs
	loss_function = model_loss_function()
	optimizer = model_optimizer(model)
	number_of_epochs = model_number_of_epochs()

	# training the model
	trained_model = training_function(model, number_of_epochs, tensors_and_iterable_training_data, 
					loss_function, optimizer)

	# validating and obtaining accuracy
	model_accuracy = validation_function(trained_model,tensors_and_iterable_training_data)
	print(f"Accuracy on the test set = {model_accuracy}")

	X_train_tensor = tensors_and_iterable_training_data[0]
	x = X_train_tensor[0]
	jitted_model = torch.jit.save(torch.jit.trace(model, (x)), "task_1a_trained_model.pth")

Epoch 1/30: Loss = 0.6024387494243425
Epoch 2/30: Loss = 0.5663721494633576
Epoch 3/30: Loss = 0.5396126488159443
Epoch 4/30: Loss = 0.5037316808412815
Epoch 5/30: Loss = 0.48391943861698283
Epoch 6/30: Loss = 0.4782300067359003
Epoch 7/30: Loss = 0.4742654757253055
Epoch 8/30: Loss = 0.4573420692106773
Epoch 9/30: Loss = 0.4460906622738674
Epoch 10/30: Loss = 0.4292291896096591
Epoch 11/30: Loss = 0.4269201760662013
Epoch 12/30: Loss = 0.41386535609590597
Epoch 13/30: Loss = 0.3973549275562681
Epoch 14/30: Loss = 0.3993378109459219
Epoch 15/30: Loss = 0.3919782628273142
Epoch 16/30: Loss = 0.38639904610041914
Epoch 17/30: Loss = 0.38489997412624033
Epoch 18/30: Loss = 0.3823326990522187
Epoch 19/30: Loss = 0.37534348127143136
Epoch 20/30: Loss = 0.37442590733026637
Epoch 21/30: Loss = 0.3681092768393714
Epoch 22/30: Loss = 0.3729730888173498
Epoch 23/30: Loss = 0.3693984834284618
Epoch 24/30: Loss = 0.36708910917413645
Epoch 25/30: Loss = 0.3616923191424074
Epoch 26/30: Loss = 0.36309